# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1:Alejandro Andres Velasco Santinato
*   Alumno 2:Daniela Leal Rivero
*   Alumno 3:






---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.


---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```


---
### 1.2. Localizar entorno de trabajo: Google colab o local

In [1]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [2]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

We're running Colab
Colab: mounting Google drive on  /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).

Colab: making sure  /content/gdrive/My Drive/08_MIAR/actividades/proyecto practico  exists.

Colab: Changing directory to  /content/gdrive/My Drive/08_MIAR/actividades/proyecto practico
/content/gdrive/My Drive/08_MIAR/actividades/proyecto practico
Archivos en el directorio: 
['old training', 'checkpoints', 'dqn_SpaceInvaders-v0_log.json', 'dqn_SpaceInvaders-v0_final_weights.h5f.index', 'dqn_SpaceInvaders-v0_final_weights.h5f.data-00000-of-00001', 'checkpoint', 'dqn_SpaceInvaders-v0_2_log.json']


---
### 1.4. Instalar librerías necesarias

NOTA: esto es un fix para colab hay que ejecutarlo y cuando pida reiniciar el entorno se reinicia, despues de reiniciarlo simplemente ejecuten lo demas y deje esto como esta

In [4]:

if IN_COLAB:
  !pip uninstall tensorflow -y
  !pip install tensorflow==2.15

Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting un

In [3]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5

else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.2.4
  %pip install tensorflow==2.5.3
  %pip install torch==2.0.1
  %pip install agents==1.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.4 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.17.3-py3-none-any.whl size=1654617 sha256=5bb158900436a3367ea538f9c691fad1114f52637896d2875a7baf8eaa8cb4c3
  Stored in directory: /root/.cache/pip/wheels/07/8b/b7/570cb90b10f17e85ccb291ba1f04af41ec697745104a2263eb
Successfully built gym
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 3.1.1
    Uninstalling cloudpickle-3.1.1:
      Successfully uninstalled cloudpickle-3.1.1
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2024.12.1 requires cloud

---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [5]:
from PIL import Image
import numpy as np
import gym
import keras
print(keras.__version__)
import tensorflow

tensorflow.keras.__version__ = keras.__version__
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

2.15.0


#### Configuración base

In [6]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [7]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

1. Implementación de la red neuronal

In [8]:
def build_model(input_shape, nb_actions):
    model = Sequential()
    model.add(Permute((2, 3, 1), input_shape=input_shape))
    model.add(Convolution2D(32, (8, 8), strides=(4, 4), activation='relu'))
    model.add(Convolution2D(64, (4, 4), strides=(2, 2), activation='relu'))
    model.add(Convolution2D(64, (3, 3), strides=(1, 1), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(nb_actions, activation='linear'))
    return model

input_shape = (WINDOW_LENGTH, INPUT_SHAPE[0], INPUT_SHAPE[1])
model = build_model(input_shape, nb_actions)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 permute (Permute)           (None, 84, 84, 4)         0         
                                                                 
 conv2d (Conv2D)             (None, 20, 20, 32)        8224      
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 64)          32832     
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 512)               1606144   
                                                                 
 dense_1 (Dense)             (None, 6)                 3

2. Implementación de la solución DQN

In [9]:
base_steps = 450000
class LinearAnnealedPolicyCustom(LinearAnnealedPolicy):
  def get_current_value(self):
    """Return current annealing value

    # Returns
        Value to use in annealing
    """
    if self.agent.training:
        # Linear annealed: f(x) = ax + b.
        a = -float(self.value_max - self.value_min) / float(self.nb_steps)
        b = float(self.value_max)
        value = max(self.value_min, a * float(self.agent.step+base_steps) + b)
    else:
        value = self.value_test
    return value

In [10]:
from tensorflow.keras.optimizers.legacy import Adam
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)


##solucionando un problema con la linea aNNEaledPolicy para retomar el trabajo

# Creando la policy
policy = LinearAnnealedPolicyCustom(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=0.05, nb_steps=1000000)
# Creando el agente de DQN
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=AtariProcessor(), nb_steps_warmup=50000, gamma=.99, target_model_update=10000,
               train_interval=4, delta_clip=1.)

# Compilando el agente
dqn.compile(Adam(learning_rate=0.00025), metrics=['mae'])

# Logica para almacenar los pesos del modelo
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=50000)]
callbacks += [FileLogger('dqn_{}_log.json'.format(env_name), interval=100)]


In [ ]:
# --- Parte de Entrenamiento con Checkpoints en Google Drive ---

# Define el directorio para los checkpoints dentro de tu carpeta de Google Drive
checkpoint_dir = os.path.join(drive_root, 'checkpoints')
# Crea el directorio si no existe (exist_ok=True evita errores si ya existe)
os.makedirs(checkpoint_dir, exist_ok=True)

# Define el nombre de archivo para guardar los pesos, incluyendo la ruta a tu carpeta de Drive
# {step} será reemplazado automáticamente por el número de pasos realizados durante el entrenamiento
checkpoint_weights_filename = os.path.join(checkpoint_dir, 'dqn_' + env_name +'_'+str(base_steps)+ '_weights_{step}.h5f')

# Define los callbacks para guardar los pesos y registrar el progreso
# ModelIntervalCheckpoint guarda los pesos en intervalos especificados
# FileLogger registra el progreso del entrenamiento en un archivo JSON
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=50000)]
callbacks += [FileLogger(os.path.join(drive_root, 'dqn_{}_2_log.json'.format(env_name)), interval=100)]


# --- Iniciar Entrenamiento ---
print("\nIniciando el entrenamiento de DQN...")
# Puedes ajustar 'nb_steps' según tus recursos computacionales y la duración deseada del entrenamiento.
# Para un buen rendimiento en Atari, esto a menudo requiere millones de pasos.
# Comienza con un número menor como 500,000 para ver el progreso, luego auméntalo.
last_checkpoint = os.path.join(checkpoint_dir, 'dqn_' + env_name + '_weights_'+str(base_steps)+'.h5f')
print(last_checkpoint)
dqn.load_weights(last_checkpoint) # Carga los pesos finales guardados anteriormente
dqn.step = base_steps
dqn.fit(env, callbacks=callbacks, nb_steps=1750000, log_interval=10000, visualize=False, verbose=2)

# Después de entrenar, guarda los pesos finales
final_weights_filename = os.path.join(drive_root, 'dqn_{}_final_weights.h5f'.format(env_name))
#cargar pesos anteriores
dqn.save_weights(final_weights_filename, overwrite=True)
print(f"\nEntrenamiento finalizado. Pesos finales guardados en: {final_weights_filename}")

# --- Parte de prueba para calcular la recompensa media (después del entrenamiento) ---
print("\nIniciando la fase de prueba...")
# Carga los pesos finales para la prueba (o un checkpoint específico si quieres probar un modelo intermedio)
dqn.load_weights(final_weights_filename) # Carga los pesos finales guardados anteriormente
dqn.test(env, nb_episodes=10, visualize=False)
print("Prueba completada.")


Iniciando el entrenamiento de DQN...
/content/gdrive/My Drive/08_MIAR/actividades/proyecto practico/checkpoints/dqn_SpaceInvaders-v0_weights_450000.h5f
Training for 1750000 steps ...
     495/1750000: episode: 1, duration: 4.421s, episode steps: 495, steps per second: 112, episode reward: 10.000, mean reward:  0.020 [ 0.000,  1.000], mean action: 2.651 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    1258/1750000: episode: 2, duration: 5.537s, episode steps: 763, steps per second: 138, episode reward: 14.000, mean reward:  0.018 [ 0.000,  1.000], mean action: 2.713 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    1641/1750000: episode: 3, duration: 3.571s, episode steps: 383, steps per second: 107, episode reward:  3.000, mean reward:  0.008 [ 0.000,  1.000], mean action: 2.663 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    2269/1750000: episode: 4, duration: 5.175s, episode steps: 628, steps per second: 121, episode reward: 11.000, me

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   50055/1750000: episode: 70, duration: 11.405s, episode steps: 614, steps per second:  54, episode reward: 15.000, mean reward:  0.024 [ 0.000,  1.000], mean action: 2.609 [0.000, 5.000],  loss: 0.012279, mae: 1.197000, mean_q: 1.463369, mean_eps: 0.549975
   50757/1750000: episode: 71, duration: 44.576s, episode steps: 702, steps per second:  16, episode reward: 13.000, mean reward:  0.019 [ 0.000,  1.000], mean action: 2.412 [0.000, 5.000],  loss: 0.015674, mae: 1.208921, mean_q: 1.474730, mean_eps: 0.549635
   51422/1750000: episode: 72, duration: 42.349s, episode steps: 665, steps per second:  16, episode reward: 12.000, mean reward:  0.018 [ 0.000,  1.000], mean action: 2.486 [0.000, 5.000],  loss: 0.015070, mae: 1.174384, mean_q: 1.428992, mean_eps: 0.549019
   52172/1750000: episode: 73, duration: 47.210s, episode steps: 750, steps per second:  16, episode reward: 15.000, mean reward:  0.020 [ 0.000,  1.000], mean action: 2.063 [0.000, 5.000],  loss: 0.014992, mae: 1.181391, m

In [11]:
# Testing part to calculate the mean reward


weights_filename = 'checkpoints/dqn_SpaceInvaders-v0_450000_weights_500000.h5f'
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Episode 1: reward: 31.000, steps: 1236
Episode 2: reward: 24.000, steps: 987
Episode 3: reward: 19.000, steps: 731
Episode 4: reward: 28.000, steps: 1061
Episode 5: reward: 17.000, steps: 616
Episode 6: reward: 21.000, steps: 968
Episode 7: reward: 26.000, steps: 961
Episode 8: reward: 10.000, steps: 503
Episode 9: reward: 14.000, steps: 705
Episode 10: reward: 11.000, steps: 486


3. Justificación de los parámetros seleccionados y de los resultados obtenidos

---